# Using Orion for Custom Data

This notebook is quick tutorial to use Orion on custom data.

## Step 1: Load your data

In [1]:
# signal from orion library that you can replace with your own data

from orion.data import load_signal

data = load_signal('S-1')

data.head()

,timestamp,value
0,1222819200,-0.366359
1,1222840800,-0.394108
2,1222862400,0.403625
3,1222884000,-0.362759
4,1222905600,-0.370746


If your data is not following the Orion standard, we need to format it such that it contains two columns:
- **timestamp**: an integer representation of time.
- **values**: the observed value of the time series at that specific time.

Format the data if necessary

In [2]:
import pandas as pd
import numpy as np

# convert the timestamp column into timestamps (integer values)

timestamps = pd.to_datetime(data['timestamp'], unit='s')
data['timestamp'] = timestamps.values.astype(np.int64) // 10 ** 9

# rename columns in the dataframe to two important columns: timestamp, and value

data = data.rename({"timestamp": "timestamp", "value": "value"})
data.head()

,timestamp,value
0,1222819200,-0.366359
1,1222840800,-0.394108
2,1222862400,0.403625
3,1222884000,-0.362759
4,1222905600,-0.370746


## Step 2: Run Orion

Use Orion to find anomalies in your time series signal.

Orion provides a collection of anomaly detection pipelines which you can choose from. You can view the pipelines and their ranking in our [leaderbord](https://github.com/sintel-dev/Orion?tab=readme-ov-file#leaderboard).

In this tutorial, we will use `AER` model.

In [3]:
from orion import Orion

hyperparameters = { # alter the hyperparameter settings here
    'mlstars.custom.timeseries_preprocessing.time_segments_aggregate#1': {
        'interval': 21600
    },
    'orion.primitives.aer.AER#1': {
        'epochs': 5,
        'verbose': True
    }
}

orion = Orion(
    pipeline='aer',
    hyperparameters=hyperparameters
)

orion.fit(data)

/opt/anaconda3/envs/orion/lib/python3.10/site-packages/sklearn/impute/_base.py:356: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


Epoch 1/5
126/126 [==============================] - 7s 32ms/step - loss: 0.1936 - tf.__operators__.getitem_loss: 0.1878 - tf.__operators__.getitem_1_loss: 0.1999 - tf.__operators__.getitem_2_loss: 0.1869 - val_loss: 0.2260 - val_tf.__operators__.getitem_loss: 0.2164 - val_tf.__operators__.getitem_1_loss: 0.2305 - val_tf.__operators__.getitem_2_loss: 0.2266
Epoch 2/5
126/126 [==============================] - 3s 27ms/step - loss: 0.1843 - tf.__operators__.getitem_loss: 0.1772 - tf.__operators__.getitem_1_loss: 0.1913 - tf.__operators__.getitem_2_loss: 0.1771 - val_loss: 0.2250 - val_tf.__operators__.getitem_loss: 0.2183 - val_tf.__operators__.getitem_1_loss: 0.2290 - val_tf.__operators__.getitem_2_loss: 0.2238
Epoch 3/5
126/126 [==============================] - 3s 28ms/step - loss: 0.1801 - tf.__operators__.getitem_loss: 0.1724 - tf.__operators__.getitem_1_loss: 0.1878 - tf.__operators__.getitem_2_loss: 0.1724 - val_loss: 0.2213 - val_tf.__operators__.getitem_loss: 0.2263 - val_tf.__o

Now we can use the trained pipeline to find anomalies:

In [4]:
orion.detect(data)

315/315 [==============================] - 1s 3ms/step


,start,end,severity
0,1400716800,1405404000,0.117529


For any questions, please open an issue on
[Orion github](https://github.com/sintel-dev/Orion)!